In [10]:
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import utils_data as ud
import utils_info as info
import utils_weather as uw
import utils_matrixes as um
import utils_regress as ur
import utils_benchmark as ub
import constants as cons
from sklearn import metrics
import statsmodels.api as sm
from pylab import rcParams
from openpyxl import load_workbook
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
sns.set(font_scale=1.2)
warnings.filterwarnings('ignore')

pd.set_option("display.max_rows", 999)

In [15]:
'''

Settings

'''
company = 'Company'
hub = 'Hub1' # 'PORELENE', 'PTOMSKEN', 'PTOMSKES'
compare_file_name = 'compare_file.xlsx'

comparison_dir = cons.BENCHMARK_FOLDER
horison = cons.FORECAST_HORISON
timestep = cons.TIMESTEP_NAME
compare_file_name = ud.get_filename(comparison_dir, company, hub, compare_file_name)
model_name= 'dense'
error_type = 'mape'             # or 'mape' or 'me'

mode = 'network' # 'network', 'regress', 'from_file'

#Do forecast inplace
min_lag = 96
start_date = pd.datetime(2019, 7, 20, 0, 0)
end_date = pd.datetime(2019, 8, 3, 23,0)
dates_range = pd.date_range(start_date - pd.Timedelta(70, unit='D'), end_date, freq='H')
check_range = pd.date_range(start_date, end_date, freq='H')
'''

Main

'''
model_info = info.get_model_info(company, hub)
actuals_data = ud.get_data(model_info['actuals_filename'], dates_range[0], 
                          dates_range[-1], custom_date_format = model_info['actuals_dateformat'])

if mode == 'from_file':

    xl = pd.ExcelFile(compare_file_name)
    cmp_df = xl.parse(model_name)
    print('This is the data of model', xl.sheet_names)
    print('from ', cmp_df.timestep[0], 'to ', cmp_df.timestep.iloc[-1])
    ud.get_accuracy(cmp_df.Fact, cmp_df.Prediction);
    
elif mode == 'network':
    
    model_settings_network = ud.load_model_settings(company, hub, min_lag, 'network')
    network_data, pred_name = um.make_network_forecast(actuals_data.copy(), model_settings_network,
                                                           model_info, dates_range[0], dates_range[-1])
    cmp_df = network_data[[cons.TIMESERIES_TARGET_NAME, 'prediction', pred_name]].copy()
            
else:    
    
    model_settings_regress = ud.load_model_settings(company, hub, min_lag, 'regress')
    regress_data, pred_name = um.make_regress_forecast(actuals_data.copy(), model_settings_regress,
                                                           model_info, dates_range[0], dates_range[-1])
    cmp_df = regress_data[[cons.TIMESERIES_TARGET_NAME, 'prediction', pred_name]].copy()
    
    
if mode == 'network' or mode == 'regress':
    
    cmp_df = cmp_df.loc[check_range, :]
    print('\nModel error WITHOUT autoregress, prediction name = prediction:')
    ud.get_accuracy(cmp_df[cons.TIMESERIES_TARGET_NAME], cmp_df.prediction);
    print('\nModel error WITH autoregress, prediction name = %s:' %(pred_name))
    ud.get_accuracy(cmp_df[cons.TIMESERIES_TARGET_NAME], cmp_df[pred_name]);
    
    print('\nEnter prediction name (prediction, %s):' % (pred_name))
    prediction_name = input()        # one from predictions

    cmp_df.rename(columns={cons.TIMESERIES_TARGET_NAME: 'Fact', prediction_name: 'Prediction'}, inplace=True)
    cmp_df[cons.TIMESTEP_NAME] = cmp_df.index    
    
    print('\nThe forecast of model %s (prediction name =  %s) will be analyzed' % (mode, prediction_name))
    print('from ', cmp_df.timestep[0], 'to ', cmp_df.timestep.iloc[-1])
    
cmp_df['mae'] = np.abs(cmp_df.Fact - cmp_df.Prediction)
cmp_df['mape'] = (np.abs((cmp_df.Fact - cmp_df.Prediction) / cmp_df.Fact)) * 100
cmp_df['me'] = cmp_df.Fact - cmp_df.Prediction

temp_df = ud.get_data(model_info['temperature_actuals_filename'], cmp_df.timestep[0], 
                          cmp_df.timestep.iloc[-1], 
                          custom_date_format = model_info['temperature_actuals_dateformat'])                      


In [ ]:
ub.mean_consumption_by_week(cmp_df)
ub.mean_temperature_by_week(temp_df)
ub.errors_by_week_line(cmp_df, error_type)
week_df = ub.errors_by_week(cmp_df, error_type)

In [ ]:
if cmp_df.shape[0] > 366*24:
    ub.mean_consumption_by_month(cmp_df)
    ub.errors_by_month_line(cmp_df, error_type)
    month_df = ub.errors_by_month(cmp_df, error_type)

In [ ]:
day_df = ub.errors_by_day(cmp_df,error_type)

In [ ]:
hour_df = ub.errors_by_hour(cmp_df, error_type)

In [ ]:
ub.plot_fact_preds(cmp_df, start_date, end_date)
ub.plot_temperature(temp_df, start_date, end_date)


In [ ]:
ub.plot_error(cmp_df, error_type)
ub.plot_hist_error(cmp_df, error_type)

In [ ]:
'''
Write to excel
'''
if mode == 'from_file':
    
    book = load_workbook(compare_file_name)
    writer = pd.ExcelWriter(compare_file_name, engine='openpyxl')
    writer.book = book
    
    ws_titles = [ws.title for ws in book.worksheets]
    
    if model_name not in ws_titles:
        
        cmp_df.to_excel(writer, sheet_name=model_name)
        
    try:    
    
        hour_df.to_excel(writer, sheet_name=model_name + '_' + error_type, startrow=0, startcol=0)
        day_df.to_excel(writer, sheet_name=model_name + '_' + error_type, startrow=0, startcol=5)
        week_df.to_excel(writer, sheet_name=model_name + '_' + error_type, startrow=0, startcol=10)
        month_df.to_excel(writer, sheet_name=model_name + '_' + error_type, startrow=0, startcol=15)
    
    except NameError:
        pass
    
    if 'Sheet1' in ws_titles:
        empty_sheet = book.get_sheet_by_name('Sheet1')
        book.remove_sheet(empty_sheet)
    
    writer.save()
    writer.close()
    
    print('Error values are loaded to ' + compare_file_name)